In [73]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [81]:
df =pd.read_csv('train_set.csv',index_col=0)
df.head(5)

,note,avis,assureur,produit,date_publication,date_exp,avis_en,tokens_en,bigrams_en,tokens_fr,bigrams_fr
0,4.0,la personne au téléphone était clair et sympat...,L'olivier Assurance,auto,06/10/2021,01/10/2021,the person on the phone was clear and friendly...,"['person', 'phone', 'clear', 'friendly', 'expl...","['person_phone', 'phone_clear', 'clear_friendl...","['personne', 'téléphone', 'clair', 'sympathiqu...","['personne_téléphone', 'téléphone_clair', 'cla..."
1,4.0,satisfaitréactivité simplicité prix attractif ...,APRIL Moto,moto,09/07/2021,01/07/2021,satisfiedreactivity simplicity attractive pric...,"['satisfiedreactivity', 'simplicity', 'attract...","['satisfiedreactivity_simplicity', 'simplicity...","['satisfaitréactivité', 'simplicité', 'prix', ...","['satisfaitréactivité_simplicité', 'simplicité..."
2,1.0,assureur à fuir n assure pas ses responsabilit...,SwissLife,vie,15/10/2020,01/10/2020,insurer to flee does not ensure its responsibi...,"['insurer', 'flee', 'ensure', 'responsibility'...","['insurer_flee', 'flee_ensure', 'ensure_respon...","['assureur', 'fuir', 'assure', 'responsabilité...","['assureur_fuir', 'fuir_assure', 'assure_respo..."
3,1.0,voilà 3 mois que la gmf me fait attendre pour ...,GMF,habitation,03/03/2020,01/03/2020,the gmf has been waiting for a water damage fo...,"['gmf', 'waiting', 'water', 'damage', 'month',...","['gmf_waiting', 'waiting_water', 'water_damage...","['voilà', 'mois', 'gmf', 'fait', 'attendre', '...","['voilà_mois', 'mois_gmf', 'gmf_fait', 'fait_a..."
4,3.0,je suis bien avec cet assuranceelle est pratiq...,L'olivier Assurance,auto,28/08/2021,01/08/2021,i am good with this insurance she is practical...,"['good', 'insurance', 'practical', 'least', 'f...","['good_insurance', 'insurance_practical', 'pra...","['bien', 'cet', 'assuranceelle', 'pratique', '...","['bien_cet', 'cet_assuranceelle', 'assuranceel..."


In [83]:
df['note'].value_counts()

1.0    7271
4.0    4885
5.0    4849
2.0    3716
3.0    3380
Name: note, dtype: int64

### Information Retrieval

In [76]:
def preprocess(text):
    # Remove special characters and put in lower case
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    # Tokenization, remove stop words, and stemming
    stop_words = set(stopwords.words('english'))
    stemmer = SnowballStemmer('english')
    tokens = [stemmer.stem(word) for word in text.split() if word not in stop_words]
    return ' '.join(tokens)

In [77]:
def retrieve_information(query, df):
    """IRS system using TF-IDF and cosine similarity to give the best results associated with the query"""
    # Combine relevant informations in one column and preprocess it
    df['information'] =  df['avis_en'] + df['produit'] + df["assureur"].str.lower()
    df['information'] = df['information'].apply(preprocess)
    
    # Use Tf_idf vectorizer for the combined_text
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['information'])

    # Vectorize the query
    query_vectorized = vectorizer.transform([preprocess(query)])
    
    # Compute cosine similarity between the query and the tfidf_matrix
    similarity_scores = cosine_similarity(query_vectorized, tfidf_matrix).flatten()

    # Add similarity scores to df and sort by similarity 
    df['similarity'] = similarity_scores
    df = df.sort_values(by='similarity', ascending=False)
    return df[['assureur', 'produit','avis_en','note','similarity']]

# Query
query = "best moto insurance"
result_df = retrieve_information(query, df)

print("Results:")
display(result_df.head(10))

Top 10 Results:


,assureur,produit,avis_en,note,similarity
21441,APRIL Moto,moto,best insurance with good quality of services a...,4.0,0.482588
1326,Direct Assurance,auto,i am really satisfied with your service top cu...,5.0,0.472610
13735,APRIL Moto,moto,i am satisfied with the price and the services...,5.0,0.448705
5688,L'olivier Assurance,auto,i am satisfied with your service prices suit m...,3.0,0.439789
10594,APRIL Moto,moto,no complaints on the prices and services offer...,4.0,0.431146
6151,L'olivier Assurance,auto,i am satisfied with the service prices suit me...,5.0,0.393101
13765,L'olivier Assurance,auto,very satisfactory service on the online phone ...,4.0,0.375686
21402,Santiane,sante,very well i find the foreign customer service ...,4.0,0.371664
5293,Direct Assurance,auto,simple and quick best price on car insurance ...,5.0,0.363730
19628,APRIL Moto,moto,i am satisfied with the service the site is ea...,4.0,0.362966
